In [1]:
#%% Set-up

import brightway2 as bw
import pandas as pd
import os
import warnings
import time
import dask

# Set working directry
path = "D:\\Andrea\\OneDrive - University College London\\S4CE\\GSA\\Geothermal"
os.chdir(path)

# Import local
from cge_klausen import parameters as cge_parameters
from ege_klausen import parameters as ege_parameters
from cge_model import GeothermalConventionalModel
from ege_model import GeothermalEnhancedModel
from s_models import simplified_cge_model, simplified_ege_model
from utils.lookup_func import lookup_geothermal
#from dev.Stoc_MultiMethod_LCA_pygsa import run_mc, run_mc_dask

# Set project
bw.projects.set_current("Geothermal")

# Retrieve methods 
ILCD_CC = [method for method in bw.methods if "ILCD 2.0 2018 midpoint no LT" in str(method) and "climate change total" in str(method)]
ILCD_HH = [method for method in bw.methods if "ILCD 2.0 2018 midpoint no LT" in str(method) and "human health" in str(method)]
ILCD_EQ = [method for method in bw.methods if "ILCD 2.0 2018 midpoint no LT" in str(method) and "ecosystem quality" in str(method)]
ILCD_RE = [method for method in bw.methods if "ILCD 2.0 2018 midpoint no LT" in str(method) and "resources" in str(method)]
ILCD = ILCD_CC + ILCD_HH + ILCD_EQ + ILCD_RE

# Find demand
_, _, _, _, _, _, _, _, _, _, _, _, _, _, electricity_conv_prod, electricity_enh_prod = lookup_geothermal()

# Number of iterations
n_iter=80

# Generate stochastic values
cge_parameters.stochastic(iterations=n_iter)

# Reference model
cge_model = GeothermalConventionalModel(cge_parameters)
cge_parameters_sto = cge_model.run_ps(cge_parameters)

# Run_mc function

## Compute normal

In [ ]:
%%time
ref_cge = run_mc(cge_parameters_sto, electricity_conv_prod, ILCD, n_iter)

## Compute dask 
## NOTE: DASK DOES NOT SEEM TO WORK WHEN IMPLEMENTED IN AN IMPORTED MODULE

In [ ]:
from dask.distributed import Client, LocalCluster

cluster = LocalCluster(n_workers=8
client = Client(cluster)


In [ ]:
client

In [ ]:
#%%time
ref_cge = run_mc_dask(cge_parameters_sto, electricity_conv_prod, ILCD[0], n_iter)


# Dask manual implementation

## Set up Dask

In [2]:
from dask.distributed import Client, LocalCluster
cluster = LocalCluster(n_workers=8)
client = Client(cluster)

In [3]:
client

Client Scheduler: tcp://127.0.0.1:53888 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 8 Cores: 8 Memory: 34.28 GB


## Define functions

In [4]:
import numpy as np
from pypardiso import spsolve

# This script peforms MonteCarlo simulations using presamples.

def find_where_in_techparams(cge_parameters_sto, lca):
    
    mask_tech = lambda inp, out: np.where(np.all([lca.tech_params['row']==inp, 
                                                  lca.tech_params['col']==out],axis=0)) [0][0]

    mask_bio  = lambda inp, out: np.where(np.all([lca.bio_params['row'] ==inp, 
                                                  lca.bio_params['col'] ==out],axis=0)) [0][0]

    n_parameters = len(cge_parameters_sto)
    where_tech = np.array([], dtype=int)
    amt_tech = np.array([])
    where_bio  = np.array([], dtype=int)
    amt_bio = np.array([])
    for j in range(n_parameters):
        if cge_parameters_sto[j][0][0] != 'biosphere3':
            input_  = lca.activity_dict[cge_parameters_sto[j][0]]
            output_ = lca.activity_dict[cge_parameters_sto[j][1]]
            temp = mask_tech(input_, output_)
            where_tech = np.append(where_tech, temp)
            amt_tech = np.append(amt_tech, cge_parameters_sto[j][2])
        else:
            input_  = lca.biosphere_dict[cge_parameters_sto[j][0]]
            output_ = lca.activity_dict[cge_parameters_sto[j][1]]
            temp = mask_bio(input_, output_)
            where_bio = np.append(where_bio, temp)
            amt_bio = np.append(amt_bio, cge_parameters_sto[j][2])
    
    if amt_tech.shape[0] != 0:
        amt_tech = amt_tech.reshape([len(where_tech),-1])
    if amt_bio.shape[0] != 0:
        amt_bio  = amt_bio.reshape([len(where_bio),-1])
    
    return where_tech, amt_tech, where_bio, amt_bio


## This function is part of run_mc, without lca object instantiation and for loop on methods

In [5]:
def replace_and_compute(lca, CF_matr, method, where_tech, amt_tech, where_bio, amt_bio,i) :
    demand_array = lca.demand_array
    tech_params = lca.tech_params['amount']
    bio_params = lca.bio_params['amount']
    
    if where_tech.shape[0] != 0:
                np.put(tech_params, where_tech, amt_tech[:,i])
                lca.rebuild_technosphere_matrix(tech_params)

    if where_bio.shape[0] != 0:
        np.put(bio_params, where_bio, amt_bio[:,i])
        lca.rebuild_biosphere_matrix(bio_params)

    score = (CF_matr*lca.biosphere_matrix) * \
             spsolve(lca.technosphere_matrix, demand_array)
    return score

## Initialize LCA object

In [6]:
demand = electricity_conv_prod
parameters=cge_parameters_sto

In [7]:
lca = bw.LCA({demand:1}, )
lca.lci()
lca.build_demand_array()
    
where_tech, amt_tech, where_bio, amt_bio = find_where_in_techparams(parameters, lca)         

## Parallelization on iterations -  Single method

In [8]:
method=ILCD[0]
lca.switch_method(method)
CF_matr = sum(lca.characterization_matrix)       
scores_m = []

In [9]:
for i in range(n_iter):
            score_m_=dask.delayed(replace_and_compute)(lca, CF_matr, method, where_tech, amt_tech, where_bio, amt_bio,i)
            scores_m.append(score_m_)

In [10]:
%%time
scores_single = {}
scores_single[method]=dask.compute(*scores_m)

Wall time: 2min 9s


In [11]:
scores_single

{('ILCD 2.0 2018 midpoint no LT',
  'climate change',
  'climate change total'): (array([0.15963449]), array([0.02898089]), array([0.22243909]), array([0.07935628]), array([0.07015773]), array([0.02072809]), array([0.06523153]), array([0.0428212]), array([0.02613432]), array([0.02120766]), array([0.21383124]), array([0.16124576]), array([0.04584552]), array([0.08050785]), array([0.20665263]), array([0.02954676]), array([0.04630818]), array([0.09466826]), array([0.42890882]), array([0.29121283]), array([0.0930545]), array([0.0926548]), array([0.03832474]), array([0.54656469]), array([0.12867711]), array([0.02675807]), array([0.13691106]), array([0.03640163]), array([0.05482605]), array([0.50580588]), array([0.13105927]), array([0.26308823]), array([0.06421962]), array([0.1894024]), array([0.01987723]), array([0.02989422]), array([0.06882307]), array([0.07419192]), array([0.07280896]), array([0.08098415]), array([0.039724]), array([0.55768712]), array([0.03278696]), array([0.05686213]), 

### No parallel computations

In [13]:
%%time
score_ref_single = []
for i in range(n_iter):
            score_m_=replace_and_compute(lca, CF_matr, method, where_tech, amt_tech, where_bio, amt_bio,i)
            score_ref_single.append(score_m_)

Wall time: 34.5 s


## Parallelization on iterations -  Multi method

In [14]:
for method in ILCD:
    lca.switch_method(method)
    CF_matr = sum(lca.characterization_matrix)       
    scores_multi = {}
    scores_multi_ = {}
    scores_m = []
    for i in range(n_iter):
            score_m_=dask.delayed(replace_and_compute)(lca, CF_matr, method, where_tech, amt_tech, where_bio, amt_bio,i)
            scores_m.append(score_m_)
    
    scores_multi_[method] = scores_m
    

In [15]:
%%time
for key in scores_multi_:
    scores_multi[key]=dask.compute(*scores_multi_[key])

Wall time: 1min 49s


In [16]:
scores_multi

{('ILCD 2.0 2018 midpoint no LT',
  'resources',
  'minerals and metals'): (array([2.25273623e-08]), array([1.65029162e-08]), array([9.24942973e-08]), array([5.10864556e-08]), array([1.95600478e-08]), array([2.31284713e-08]), array([1.690699e-07]), array([4.35788175e-08]), array([2.70825099e-08]), array([3.85866081e-08]), array([1.99194427e-07]), array([2.86369356e-08]), array([2.1502465e-08]), array([3.13731112e-08]), array([5.52867026e-08]), array([1.70899203e-08]), array([1.58380221e-08]), array([3.3737955e-08]), array([7.49593547e-08]), array([1.80097368e-08]), array([1.48050911e-08]), array([6.27680304e-08]), array([1.11573634e-07]), array([2.17813559e-08]), array([1.634373e-08]), array([1.79815577e-08]), array([3.4847441e-08]), array([5.28875727e-08]), array([4.2149326e-08]), array([2.29210034e-08]), array([1.95736658e-08]), array([2.19058471e-08]), array([5.27938848e-08]), array([2.54960564e-08]), array([1.97445362e-08]), array([3.27463051e-08]), array([4.78332889e-08]), array([


# Parallelization on methods - Multi-method


In [17]:
# This function implements a for loop on n_iter using the previously defined replace_and_compute
def replace_and_compute_method (lca, CF_matr, where_tech, amt_tech, where_bio, amt_bio, n_iter):     
    scores_m = []
    for i in range(n_iter):
            score_m_=replace_and_compute(lca, CF_matr, method, where_tech, amt_tech, where_bio, amt_bio,i)
            scores_m.append(score_m_)
    return scores_m

In [18]:
# We need to get this out of the dask delayed otherwise in does not work!
def get_CF_matr (lca, method):
    lca.switch_method(method)
    CF_matr = sum(lca.characterization_matrix)
    return CF_matr

In [19]:
CF_matr ={}
for method in ILCD:
    CF_matr[method] = get_CF_matr(lca, method)

In [20]:
%%time
scores_2={}
for method in ILCD:
    scores_2[method]=replace_and_compute_method(lca, CF_matr[method], where_tech, amt_tech, where_bio, amt_bio, n_iter)
     

Wall time: 9min 3s


In [21]:
scores_2_delayed=[]
for method in ILCD:
    scores_2_delayed.append( dask.delayed(replace_and_compute_method)(lca, CF_matr[method], where_tech, amt_tech, where_bio, amt_bio, n_iter) )

In [22]:
%%time
dask.compute(*scores_2_delayed)

C:\Users\Andrea\Miniconda3\envs\brightway\lib\site-packages\distributed\worker.py:3285: UserWarning: Large object of size 34.88 MB detected in task graph: 
  (<bw2calc.lca.LCA object at 0x000002EF8FEE2B38>, < ... 8079616]]), 80)
Consider scattering large objects ahead of time
with client.scatter to reduce scheduler burden and 
keep data on workers

    future = client.submit(func, big_data)    # bad

    big_future = client.scatter(big_data)     # good
    future = client.submit(func, big_future)  # good
  % (format_bytes(len(b)), s)


Wall time: 16min 32s


([array([0.15963449]),
  array([0.02898089]),
  array([0.22243909]),
  array([0.07935628]),
  array([0.07015773]),
  array([0.02072809]),
  array([0.06523153]),
  array([0.0428212]),
  array([0.02613432]),
  array([0.02120766]),
  array([0.21383124]),
  array([0.16124576]),
  array([0.04584552]),
  array([0.08050785]),
  array([0.20665263]),
  array([0.02954676]),
  array([0.04630818]),
  array([0.09466826]),
  array([0.42890882]),
  array([0.29121283]),
  array([0.0930545]),
  array([0.0926548]),
  array([0.03832474]),
  array([0.54656469]),
  array([0.12867711]),
  array([0.02675807]),
  array([0.13691106]),
  array([0.03640163]),
  array([0.05482605]),
  array([0.50580588]),
  array([0.13105927]),
  array([0.26308823]),
  array([0.06421962]),
  array([0.1894024]),
  array([0.01987723]),
  array([0.02989422]),
  array([0.06882307]),
  array([0.07419192]),
  array([0.07280896]),
  array([0.08098415]),
  array([0.039724]),
  array([0.55768712]),
  array([0.03278696]),
  array([0.056862

### No parallel computations

In [27]:
%%time
for method in ILCD:
    lca.switch_method(method)
    CF_matr = sum(lca.characterization_matrix)       
    scores_ref_multi = {}
    scores_m_ = {}
    scores_m = []
    for i in range(n_iter):
            score_m_=replace_and_compute(lca, CF_matr, method, where_tech, amt_tech, where_bio, amt_bio,i)
            scores_m.append(score_m_)
    scores_ref_multi[method] = scores_m
    

Wall time: 9min 45s
